In [1]:

from sklearn.metrics import r2_score
import copy
import operator
from tensorflow.keras.losses import BinaryCrossentropy
import random
import pandas as pd
import keras.metrics as metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Flatten,Dense,Input,Reshape
from tensorflow.keras.models import Sequential
from random import randrange



In [2]:
from genetic_algorithm_net import GA

### Import dataset

In [3]:
df = pd.read_csv('../Data/processed_df.csv')

In [4]:
df.head()

,handgrip,weight_loss,exaustion,poor_endurance,gait_speed,balance,hearing_deficit,distance_vision,near_vision,depression_scale,...,idade_morador_8,idade_morador_9,idade_morador_10,idade_morador_11,idade_morador_12,idade_morador_13,idade_morador_14,adl_index,aadl_index,iadl_index
0,24.333333,0.0,1.0,1.0,3.500,4.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
1,25.666667,0.0,1.0,1.0,3.300,4.0,4.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,8.0
2,14.666667,NaN,2.0,1.0,4.595,4.0,2.0,3.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
3,24.000000,0.0,1.0,1.0,5.610,4.0,3.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,8.0
4,NaN,0.0,4.0,2.0,38.145,NaN,3.0,2.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,3.0


In [5]:
df.shape

(9412, 63)

### Remove missing data

In [6]:
ci_map = {
    'cognitive': ['semantic_memory', 'verbal_fluency', 'memory_recall', 'temporal_orientation'],
    'psychologicao': ['depression_scale', 'sleep_quality'],
    'locomotor': ['gait_speed', 'balance'],
    'vitality': ['handgrip', 'poor_endurance', 'weight_loss', 'exaustion'],
    'sensory': ['distance_vision', 'near_vision', 'hearing_deficit']
}

ci_cols = []
[ci_cols.extend(value) for value in ci_map.values()]
ci_cols = ci_cols + ['iadl_index']
ci_cols

['semantic_memory',
 'verbal_fluency',
 'memory_recall',
 'temporal_orientation',
 'depression_scale',
 'sleep_quality',
 'gait_speed',
 'balance',
 'handgrip',
 'poor_endurance',
 'weight_loss',
 'exaustion',
 'distance_vision',
 'near_vision',
 'hearing_deficit',
 'iadl_index']

In [7]:
df_clean = df.dropna(subset=ci_cols)
df_clean.shape

(6916, 63)

In [8]:
df_clean['iadl_index'].value_counts()

iadl_index
8.0    5860
7.0     659
6.0     212
5.0     101
4.0      46
3.0      25
2.0       6
0.0       4
1.0       3
Name: count, dtype: int64

In [9]:
# def calc_iadl(x):
#     if x == 8:
#         return 0
#     elif x == 7:
#         return 1
#     else:
#         return 2


def calc_iadl(x):
    if x == 8:
        return 0
    elif x == 7:
        return 1
    else:
        return 2

df_clean['iadl_index'] = df_clean['iadl_index'].map(lambda x: calc_iadl(x))

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_8092\3209756502.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['iadl_index'] = df_clean['iadl_index'].map(lambda x: calc_iadl(x))


In [10]:
df_clean['iadl_index'].value_counts()

iadl_index
0    5860
1     659
2     397
Name: count, dtype: int64

In [11]:
df_clean['iadl_index']


0       0
1       0
3       0
5       2
7       2
       ..
9403    0
9404    0
9405    0
9407    0
9411    0
Name: iadl_index, Length: 6916, dtype: int64

### Split Train and test


In [12]:
import pickle
# Define dataset
df_autoencoder = df_clean[ci_cols]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_autoencoder.drop(columns=['iadl_index']), df_autoencoder['iadl_index'], test_size=0.2, random_state=42)

In [13]:
X_train.shape, y_test.shape

((5532, 15), (1384,))

In [14]:
X_train = X_train.reset_index(drop=True)

y_train = y_train.reset_index(drop=True)

In [15]:

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)




with open('./out/autoencoder_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
X_train_scaled.shape, X_test_scaled.shape

((5532, 15), (1384, 15))

In [16]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=42, sampling_strategy = 'all')
X_train_scaled_over, y_train_over = ros.fit_resample(X_train_scaled, y_train)

#### Run GA

In [17]:
DNA_parameter = [["tanh","softmax","relu","sigmoid","linear"],
                 ["sgd","adam"]]



In [18]:
ga = GA(coding_size=2, X_train=X_train_scaled, X_test=[], y_train=np.array(y_train), y_test=[], DNA_parameter=DNA_parameter, epochs=10)

In [19]:
pop = ga.create_population(hidden_layers_encoder=2, hidden_layers_decoder=3, max_dim_increase=10, population_size=20)

In [20]:
ga.print_architecture(pop[5])

------------------------------
15 None
3 sigmoid
2 tanh
3 softmax
optimizer adam
------------------------------


In [21]:
ga.train(ga.build_neural_net(pop[5]))

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8398 - loss: 0.4867
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8432 - loss: 0.4752
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8353 - loss: 0.4894
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8156 - loss: 0.5100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8325 - loss: 0.4875


(0.8463486671447754, 0, <Functional name=functional_1, built=True>)

In [22]:
best_arch = ga.GA(population=pop, n_generations=2)

Generation: 0


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8398 - loss: 0.5474
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8432 - loss: 0.5402
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8353 - loss: 0.5593
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8156 - loss: 0.6059
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8321 - loss: 0.5654
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8398 - loss: 0.5420
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8432 - loss: 0.5288
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8353 - loss: 0.5364
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8156 - loss: 0.5674
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8321 - loss: 0.5254
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8398 - loss: 0.5239
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8432 - loss: 0.4811
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8353 - loss: 0.4984
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 

Best: 
------------------------------
15 None
9 sigmoid
2 tanh
3 softmax
optimizer adam
------------------------------
Second Best: 
------------------------------
15 None
3 sigmoid
2 sigmoid
3 softmax
optimizer sgd
------------------------------
Loser: 
------------------------------
15 None
3 sigmoid
2 sigmoid
3 softmax
optimizer sgd
------------------------------
Best Child: 
------------------------------
15 None
2 tanh
3 softmax
optimizer adam
------------------------------
Second Best Child: 
------------------------------
15 None
9 sigmoid
3 sigmoid
2 tanh
3 softmax
optimizer adam
------------------------------
Best result in generation (f1) 0
Best result in generation (acc) 0.8494221448898316
{'p': {'encoder': [<genetic_algorithm_net.architecture object at 0x00000227B8F4F520>, <genetic_algorithm_net.architecture object at 0x00000227B8F4F5B0>], 'code': [<genetic_algorithm_net.architecture object at 0x00000227B8F4F610>], 'decoder': [<genetic_algorithm_net.architecture object at 0

### Import Genetic Result

In [23]:
model = tf.keras.models.load_model('./out/best_model.h5')

In [24]:
model.predict(X_test_scaled)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[0.5948587 , 0.2598636 , 0.14527778],
       [0.4561975 , 0.19872366, 0.34507886],
       [0.7019341 , 0.19916928, 0.09889654],
       ...,
       [0.9365731 , 0.04946097, 0.01396598],
       [0.9297402 , 0.0553677 , 0.01489207],
       [0.82170355, 0.12802483, 0.05027158]], dtype=float32)

In [ ]:
from sklearn.metrics import r2_score

decoded_data = model.predict(X_test_scaled)
global_r2 = r2_score(X_test_scaled, decoded_data)
print(global_r2)